In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pyart
import glob
import os
from pyart.io.common import radar_coords_to_cart
from skewt import SkewT
from datetime import datetime
from csu_radartools import (csu_fhc, csu_liquid_ice_mass, csu_blended_rain, 
                            csu_dsd, csu_kdp, csu_misc)
%matplotlib inline

First define a function to plot two panels of radar data next to each other.

In [2]:
def two_panel_plot(radar, sweep=0, var1='reflectivity', vmin1=0, vmax1=65, cmap1='RdYlBu_r', 
                   units1='dBZ', var2='differential_reflectivity', vmin2=-5, vmax2=5, 
                   cmap2='RdYlBu_r', units2='dB', return_flag=False, xlim=[-150,150],
                   ylim=[-150,150]):
    display = pyart.graph.RadarDisplay(radar)
    fig = plt.figure(figsize=(13,5))
    ax1 = fig.add_subplot(121)
    display.plot_ppi(var1, sweep=sweep, vmin=vmin1, vmax=vmax1, cmap=cmap1, 
                     colorbar_label=units1, mask_outside=True)
    display.set_limits(xlim=xlim, ylim=ylim)
    ax2 = fig.add_subplot(122)
    display.plot_ppi(var2, sweep=sweep, vmin=vmin2, vmax=vmax2, cmap=cmap2, 
                     colorbar_label=units2, mask_outside=True)
    display.set_limits(xlim=xlim, ylim=ylim)
    if return_flag:
        return fig, ax1, ax2, display

Make a function to easily add a field to the radar object with all the bells and whistles (attributes) associated with it.

In [3]:
def add_field_to_radar_object(field, radar, field_name='FH', units='unitless', 
                              long_name='Hydrometeor ID', standard_name='Hydrometeor ID',
                              dz_field='ZC'):
    """
    Adds a newly created field to the Py-ART radar object. If reflectivity is a masked array,
    make the new field masked the same as reflectivity.
    """
    fill_value = -32768
    masked_field = np.ma.asanyarray(field)
    masked_field.mask = masked_field == fill_value
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', 
                np.logical_or(masked_field.mask, radar.fields[dz_field]['data'].mask))
        fill_value = radar.fields[dz_field]['_FillValue']
    field_dict = {'data': masked_field,
                  'units': units,
                  'long_name': long_name,
                  'standard_name': standard_name,
                  '_FillValue': fill_value}
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

Start by reading in the data with PyARt. This is pretty straightfoward with a simple:
    
    radar = pyart.io.read(radarfile)
    
We will also read in a sounding (here from the University of Wyoming) using SkewT.

In [4]:
#Read in the data
sndfile = '/Users/bdolan/scratch/PyART-demo/Sounding_LMN_2011052000.txt'
radarfile = '/Users/bdolan/scratch/PyART-demo/' + \
    '235545.mdv'
dformat = "%Y%m%d_%H%M%S"

##Python nicely reads time formats with the datetime module.
cdates=datetime.strptime(os.path.basename(radarfile),'%H%M%S.mdv')

radar = pyart.io.read(radarfile)
sounding = SkewT.Sounding(sndfile)

time1= '20110523_'+cdates.strftime('%H%M%S')
time1

'20110523_235545'

Now let's look at what's in the file. Py-Art reads the data into a very large object which has many attributes. A really useful way to see a bunch of information is with radar.info('compact') (or 'standard' or 'full')

In [5]:
radar.info('compact')   # see what happens with 'standard' or 'full'

altitude: <ndarray of type: float64 and shape: (1,)>
altitude_agl: None
antenna_transition: None
azimuth: <ndarray of type: float64 and shape: (6120,)>
elevation: <ndarray of type: float64 and shape: (6120,)>
fields:
	differential_phase: <ndarray of type: float32 and shape: (6120, 983)>
	cross_correlation_ratio: <ndarray of type: float32 and shape: (6120, 983)>
	normalized_coherent_power: <ndarray of type: float32 and shape: (6120, 983)>
	spectrum_width: <ndarray of type: float32 and shape: (6120, 983)>
	reflectivity: <ndarray of type: float32 and shape: (6120, 983)>
	differential_reflectivity: <ndarray of type: float32 and shape: (6120, 983)>
	specific_differential_phase: <ndarray of type: float32 and shape: (6120, 983)>
	velocity: <ndarray of type: float32 and shape: (6120, 983)>
fixed_angle: <ndarray of type: float32 and shape: (17,)>
instrument_parameters:
	radar_beam_width_h: <ndarray of type: float32 and shape: (1,)>
	unambiguous_range: <ndarray of type: float32 and shape: (6120,

Now we can extract some useful information. What's the Nyquist velocity?

In [6]:
radar.get_nyquist_vel(0,check_uniform=True)

16.525999069213867

In [7]:
radar.range['data'][0:3]

array([ 117.87839508,  237.79537964,  357.71237183], dtype=float32)

In [15]:
radar.range.keys()

['comment',
 'long_name',
 'standard_name',
 'meters_to_center_of_first_gate',
 'meters_between_gates',
 'units',
 'data',
 'spacing_is_constant',
 'axis']

In [22]:
radar.range['units']

'meters'

What about the latitude and longitude?

In [ ]:
radar.longitude['data'][0]


In [ ]:
radar.latitude['data'][0]

Ok, what does this data even look like? Luckily, Py-Art has build in plotting routines.

In [ ]:
lim=[-120,120]

two_panel_plot(radar, sweep=0, var1='reflectivity', var2='velocity', vmin1=0, vmax1=60, vmin2=-25, vmax2=25, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='dBZ', 
               units2='m s-2', xlim=lim, ylim=lim)

In [ ]:

two_panel_plot(radar, sweep=4, var1='differential_reflectivity', var2='cross_correlation_ratio', vmin1=-4, vmax1=6, vmin2=0.0, vmax2=1, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='dBZ', 
               units2='m s-2', xlim=lim, ylim=lim)

In [ ]:
lim=[-120,120]


two_panel_plot(radar, sweep=4, var1='spectrum_width', var2='differential_phase', vmin1=0, vmax1=6, vmin2=-180, vmax2=0, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='dBZ', 
               units2='m s-2', xlim=lim, ylim=lim)

Hmmm. So data is a bit of a mess. Let's clean up some.

In [ ]:
lim=[-120,120]
two_panel_plot(radar, sweep=4, var1='normalized_coherent_power', var2='specific_differential_phase', vmin1=0, vmax1=1, vmin2=-2, vmax2=6, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='dBZ', 
               units2='m s-2', xlim=lim, ylim=lim)

In [ ]:
NC = radar.fields['normalized_coherent_power']['data']

Let's mask the data based on the normalized coherent power

In [ ]:
is_messy = NC < 0.6

# Now to Numpy's masked array.. the np.ma module has a variety of methods.. here
# we will use the masked_where method. This is invoked:
# masked_where(condition, data)
# and everywhere where condition = True will be marked as masked in the resultant array
dz_masked = np.ma.masked_where(is_messy, radar.fields['reflectivity']['data'])
nc_masked = np.ma.masked_where(is_messy, radar.fields['normalized_coherent_power']['data'])


Now add a field back into the radar structure 'corrected_reflectivity'

In [ ]:
field_dict = {'data' : dz_masked}
for key in ['_FillValue', 'long_name', 'units', 
            'standard_name', 'coordinates']:
    field_dict.update({key : radar.fields['reflectivity'][key]})

# and BOOM! Just add it back into our radar object! 
radar.add_field('corrected_reflectivity', field_dict)  

In [ ]:
lim=[-120,120]

two_panel_plot(radar, sweep=4, var1='corrected_reflectivity', var2='normalized_coherent_power', vmin1=0, vmax1=60, vmin2=0, vmax2=1, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='dBZ', 
               units2='m s-2', xlim=lim, ylim=lim)

Now we can apply that mask to the rest of our fields to work with.

In [ ]:
radar.fields['velocity']['data'].mask = np.ma.mask_or(radar.fields['corrected_reflectivity']['data'].mask, radar.fields['velocity']['data'].mask)
radar.fields['differential_phase']['data'].mask = np.ma.mask_or(radar.fields['corrected_reflectivity']['data'].mask,radar.fields['differential_phase']['data'].mask)
radar.fields['differential_reflectivity']['data'].mask = np.ma.mask_or(radar.fields['corrected_reflectivity']['data'].mask,radar.fields['differential_reflectivity']['data'].mask)
radar.fields['cross_correlation_ratio']['data'].mask = np.ma.mask_or(radar.fields['corrected_reflectivity']['data'].mask,radar.fields['cross_correlation_ratio']['data'].mask)





**Unfolding Radial Velocity**

Remember the folded radial velocities? Let's try to unfold them with PyART.

In [ ]:
#### Now for the unfolding.

gatefilter = pyart.filters.GateFilter(radar)
gatefilter.exclude_invalid('corrected_reflectivity')
gatefilter.exclude_invalid('velocity')




There are actually **3** ways to unfold velocities in PyART:
    
    1.pyart.correct.dealias_region_based
    2.pyart.correct.dealias_unwrap_phase
    3.pyart.correct.dealias_fourdd
    
Let's just try the region based as an example.

In [ ]:
dealias_data_region=pyart.correct.dealias_region_based(radar,skip_between_rays=100, skip_along_ray=100, 
                                                       centered=True, nyquist_vel=16.5, check_nyquist_uniform=True, 
                                                       gatefilter=gatefilter, rays_wrap_around=None, 
                                                       keep_original=False, set_limits=False, vel_field='velocity', 
                                                       corr_vel_field=None)

radar.add_field('corrected_velocity1', dealias_data_region)



In [ ]:
lim=[-120,120]

two_panel_plot(radar, sweep=2, var1='velocity', var2='corrected_velocity1', vmin1=-25, vmax1=25, vmin2=-25, vmax2=25, 
               cmap1='RdYlBu_r', cmap2='RdYlBu_r', units1='m s-1', 
               units2='m s-1', xlim=lim, ylim=lim)

Ok. Obviously there are many, many more things you can do with PyArt, but hopefully this gets you started. A useful resource is http://arm-doe.github.io/pyart/dev/auto_examples/index.html

**Let's move on to CSU-Radartools**

The way CSU_RadarTools is designed is it works on Python arrays or scalars. This allows you to use it with any kind of radar data, whether it was ingested via Py-ART or something else, or is in polar coordinate or gridded form. Thus, we first need to extract the relevant fields from the Py-ART radar object.

In [ ]:
dz = radar.fields['corrected_reflectivity']['data']
dr = radar.fields['differential_reflectivity']['data']
ph = radar.fields['differential_phase']['data']
rh = radar.fields['cross_correlation_ratio']['data']

**csu_kdp**

This module supplies a simple way of estimating specific differential phase (Kdp) via a methodology developed in the CSU Department of Electrical Engineering, and then subsequently adapted and used in the Department of Atmospheric Science. Filtering of the input differential phase field is based on a finite impulse response (FIR) filter, applied in a moving 3-km window. Then, an adaptive linear fit is applied iteratively to the filtered phase field, where half the slope of the linear fit at any specific gate is the Kdp estimate. The length of the line needed (i.e., number of gates considered) depends on the reflectivity at the gate in question.
Standard deviation of differential phase is estimated and used to remove noisy/bad data. Gaps are filled in the filter window if the holes are less than 20% of the window length. To use this module you need a gate spacing that divdes evenly into 3 km (e.g., 50, 100, 150, 250, 300 m, etc.).

In [ ]:
# Range needs to be supplied as a variable, and it needs to be the same shape as dzN, etc.
rng2d, az2d = np.meshgrid(radar.range['data'], radar.azimuth['data'])

In [ ]:
np.shape(rng2d)


OK, now we have all our needed inputs to calculate Kdp. The function we will call is csu_kdp.calc_kdp_bringi(), and it returns Kdp, filtered differential phase, and standard deviation of differential phase, in that order. Input variables can be 1D (rays) or 2D (azimuth/elevation and rays). The fundamental algorithm works on a ray-by-ray basis.

In [ ]:
kdN, fdN, sdN = csu_kdp.calc_kdp_bringi(
    dp=ph, dz=dz, rng=rng2d/1000.0, thsd=12, gs=150.0, window=3)

In [ ]:
radar = add_field_to_radar_object(kdN, radar, field_name='KDP', units='deg/km', 
                                   long_name='Specific Differential Phase',
                                   standard_name='Specific Differential Phase', 
                                   dz_field='corrected_reflectivity')
radar = add_field_to_radar_object(fdN, radar, field_name='FDP', units='deg', 
                                   long_name='Filtered Differential Phase',
                                   standard_name='Filtered Differential Phase', 
                                   dz_field='corrected_reflectivity')
radar = add_field_to_radar_object(sdN, radar, field_name='SDP', units='deg', 
                                   long_name='Standard Deviation of Differential Phase',
                                   standard_name='Standard Deviation of Differential Phase', 
                                   dz_field='corrected_reflectivity')

In [ ]:
#First let's see what the original data looked like.
limN = [-120, 120]
two_panel_plot(radar, sweep=1, var1='corrected_reflectivity', vmin1=0, vmax1=65.0, 
               cmap1='RdYlBu_r', units1='dBZ',
               var2='differential_phase', vmin2=-180, vmax2=180, 
               cmap2='cubehelix', units2='deg', 
               xlim=limN, ylim=limN)

In [ ]:
#Now let's see the filtered and specific differential phase fields
two_panel_plot(radar, sweep=1, var1='FDP', vmin1=-180, vmax1=180, 
               cmap1='cubehelix', units1='deg',
               var2='KDP', vmin2=-5, vmax2=5, cmap2='RdYlBu_r', units2='deg/km', 
               xlim=limN, ylim=limN)

Now that we have Kdp, we have all the variables necessary for hydrometeor identification.

**csu_fhc**

I lied. First we need to get Temperature from the sounding into the same array size as the other fields. Here is a way to do that via Py-ART and numpy.interp().

In [ ]:
def get_z_from_radar(radar):
    """Input radar object, return z from radar (km, 2D)"""
    azimuth_1D = radar.azimuth['data']
    elevation_1D = radar.elevation['data']
    srange_1D = radar.range['data']
    sr_2d, az_2d = np.meshgrid(srange_1D, azimuth_1D)
    el_2d = np.meshgrid(srange_1D, elevation_1D)[1]
    xx, yy, zz = radar_coords_to_cart(sr_2d/1000.0, az_2d, el_2d)
    return zz + radar.altitude['data']

def check_sounding_for_montonic(sounding):
    """
    So the sounding interpolation doesn't fail, force the sounding to behave
    monotonically so that z always increases. This eliminates data from
    descending balloons.
    """
    snd_T = sounding.soundingdata['temp']  # In old SkewT, was sounding.data
    snd_z = sounding.soundingdata['hght']  # In old SkewT, was sounding.data
    dummy_z = []
    dummy_T = []
    if not snd_T.mask[0]: #May cause issue for specific soundings
        dummy_z.append(snd_z[0])
        dummy_T.append(snd_T[0])
        for i, height in enumerate(snd_z):
            if i > 0:
                if snd_z[i] > snd_z[i-1] and not snd_T.mask[i]:
                    dummy_z.append(snd_z[i])
                    dummy_T.append(snd_T[i])
        snd_z = np.array(dummy_z)
        snd_T = np.array(dummy_T)
    return snd_T, snd_z

def interpolate_sounding_to_radar(sounding, radar):
    """Takes sounding data and interpolates it to every radar gate."""
    radar_z = get_z_from_radar(radar)
    radar_T = None
    snd_T, snd_z = check_sounding_for_montonic(sounding)
    shape = np.shape(radar_z)
    rad_z1d = radar_z.ravel()
    rad_T1d = np.interp(rad_z1d, snd_z, snd_T)
    return np.reshape(rad_T1d, shape), radar_z



In [ ]:
radar_T, radar_z = interpolate_sounding_to_radar(sounding, radar)

To quote TJL from the CSU_radartools notebook, 'Behold!'

In [ ]:
scores = csu_fhc.csu_fhc_summer(dz=dz, zdr=dr, rho=rh, kdp=kdN, use_temp=True, band='C',
                                T=radar_T)
fh = np.argmax(scores, axis=0) + 1

To enable the ability to find out the second-ranked (or third, etc.) species, csu_fhc_summer() returns the scores for all the different categories, not just the max. So to get the traditional HID category number you have to use numpy.argmax() as above. The summer HID from CSU returns 10 possible categories:
1. Drizzle
2. Rain
3. Ice Crystals
4. Aggregates
5. Wet/Melting Snow
6. Vertically Aligned Ice
7. Low-Density Graupel
8. High-Density Graupel
9. Hail
10. Big Drops
And these are represented as integers in the newly created fh array, which as the same structure as dz, dr, etc. We'd like to plot these data using Py-ART, which means we need to turn fh in a radar object field. Let's do that.

In [ ]:
radar = add_field_to_radar_object(fh, radar,dz_field='corrected_reflectivity')

Now we need to set up a colorbar and labels to work nicely with PyART for discrete fields.

In [ ]:
hid_colors = ['White', 'LightBlue', 'MediumBlue', 'DarkOrange', 'LightPink',
              'Cyan', 'DarkGray', 'Lime', 'Yellow', 'Red', 'Fuchsia']
cmaphid = colors.ListedColormap(hid_colors)

def adjust_fhc_colorbar_for_pyart(cb):
    cb.set_ticks(np.arange(1.4, 10, 0.9))
    cb.ax.set_yticklabels(['Drizzle', 'Rain', 'Ice Crystals', 'Aggregates',
                           'Wet Snow', 'Vertical Ice', 'LD Graupel',
                           'HD Graupel', 'Hail', 'Big Drops'])
    cb.ax.set_ylabel('')
    cb.ax.tick_params(length=0)
    return cb



In [ ]:
# Actual plotting done here
lim = [-120, 120]
fig, ax1, ax2, display = two_panel_plot(radar, sweep=2, var1='corrected_reflectivity', var2='FH', vmin2=0, 
                                        vmax2=10, cmap2=cmaphid, units2='', return_flag=True, 
                                        xlim=lim, ylim=lim)
display.cbs[1] = adjust_fhc_colorbar_for_pyart(display.cbs[1])

The HID algorithm, like most algorithms in CSU_RadarTools, works on scalars too. So if you are just curious what category a set of polarimetric values will get, try the following:

In [ ]:
scores = csu_fhc.csu_fhc_summer(dz=45.0, zdr=0.0, kdp=-0.2, rho=0.95, T=-1) 
print(np.argmax(scores, axis=0) + 1)

Which is high density graupel. Pretty easy, eh?

**There are several other tools that work in a similar framework.**

*csu_fhc - fuzzy-logic hydrometeor identification

*csu_liquid_ice_mass - liquid/ice water mass calculations

*csu_blended_rain - rainfall estimation via the famous CSU blended algorithm

*csu_dsd - DSD parameter estimation via several different methodologies

*csu_kdp - An FIR-based KDP estimation algorithm

*csu_misc - A loose collection of miscellaneous tools, mainly focused on QC

**Let's do a few more things.**

First, write out a cfradial file with the radar object.

In [ ]:
pyart.io.cfradial.write_cfradial('CSAPR_corr_{t1:}_raw.nc'.format(t1=time1), radar, format='NETCDF4', time_reference=True, arm_time_variables=False)

Now, let's try **gridding** the data.

In [ ]:
grids = pyart.map.grid_from_radars(
         (radar,), grid_shape=(52, 401, 401),
        grid_limits=((0, 17000.0),(-120000, 120000), (-120000, 120000)),
        fields=radar.fields.keys(), gridding_algo="map_gates_to_grid",
        weighting_function='CRESSMAN')

In [ ]:
display = pyart.graph.GridMapDisplay(grids)
fig = plt.figure(figsize=[15, 7])

# panel sizes
map_panel_axes = [0.05, 0.05, .5, .80]
x_cut_panel_axes = [0.58, 0.10, .4, .25]
y_cut_panel_axes = [0.58, 0.50, .4, .25]

# parameters
level = 3 #21
vmin = -8
vmax = 64
lat = 36.1
lon = -97.450546264648438

# panel 1, basemap, radar reflectivity and NARR overlay
ax1 = fig.add_axes(map_panel_axes)
display.plot_basemap(lon_lines = np.arange(-104, -93, 1) )
display.plot_grid('corrected_reflectivity', level=level, vmin=vmin, vmax=vmax,
                 cmap = pyart.graph.cm.NWSRef)
display.plot_crosshairs(lon=lon, lat=lat)
#vmin = -25
#vmax = 25


# panel 2, longitude slice.
ax2 = fig.add_axes(x_cut_panel_axes)
display.plot_longitude_slice('corrected_reflectivity', lon=lon, lat=lat, vmin=vmin, vmax=vmax,
                            cmap = pyart.graph.cm.NWSRef)
ax2.set_ylim([0,17])
ax2.set_xlim([-120,120])

ax2.set_xlabel('Distance from CSAPR (km)')
vmin = -2
vmax = 6


# panel 3, latitude slice
ax3 = fig.add_axes(y_cut_panel_axes)
ax3.set_ylim([0,17])
ax3.set_xlim([-120,120])

display.plot_latitude_slice('KDP', lon=lon, lat=lat, vmin=vmin, vmax=vmax,
                           cmap = pyart.graph.cm.NWSRef)